## 불균형 데이터셋에 대한 웨이퍼 불량식별을 위한 CNN
- 모델 구성
    - Batch Normalization
    - Spatical Dropout
    - Regularization

In [1]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import random
import datetime
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
wm811k = pd.read_pickle('./data/LSWMD.pkl')

In [3]:
wm811k.head()

,waferMap,dieSize,lotName,waferIndex,trianTestLabel,failureType
0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,1.0,[[Training]],[[none]]
1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,2.0,[[Training]],[[none]]
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,3.0,[[Training]],[[none]]
3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,4.0,[[Training]],[[none]]
4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,5.0,[[Training]],[[none]]


In [4]:
wm811k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 811457 entries, 0 to 811456
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   waferMap        811457 non-null  object 
 1   dieSize         811457 non-null  float64
 2   lotName         811457 non-null  object 
 3   waferIndex      811457 non-null  float64
 4   trianTestLabel  811457 non-null  object 
 5   failureType     811457 non-null  object 
dtypes: float64(2), object(4)
memory usage: 37.1+ MB


In [5]:
# 불필요 컬럼 제거
wm811k = wm811k.drop(['waferIndex'], axis = 1)
wm811k.head()

,waferMap,dieSize,lotName,trianTestLabel,failureType
0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,[[Training]],[[none]]
1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,[[Training]],[[none]]
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,[[Training]],[[none]]
3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,[[Training]],[[none]]
4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,[[Training]],[[none]]


In [6]:
# wafermap size 확인 및 컬럼 추가
# def find_dim(x):
#     dim0=np.size(x,axis=0)
#     dim1=np.size(x,axis=1)
#     return dim0,dim1
# wm811k['waferMapDim']=wm811k['waferMap'].apply(find_dim)
# wm811k.head()

In [7]:
# 불량 클래스 확인 및 학습/검증/테스트 데이터 셋 확인
wm811k['failureNum']=wm811k['failureType']
wm811k['trainTestNum']=wm811k['trianTestLabel']
mapping_type={'Center':0,'Donut':1,'Edge-Loc':2,'Edge-Ring':3,'Loc':4,'Random':5,'Scratch':6,'Near-full':7,'none':8}
mapping_traintest={'Training':0,'Test':1}
wm811k=wm811k.replace({'failureNum':mapping_type, 'trainTestNum':mapping_traintest})
wm811k.head()

C:\Users\kjt\Anaconda3\envs\dlit\lib\site-packages\pandas\core\array_algos\replace.py:60: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  op = lambda x: operator.eq(x, b)


,waferMap,dieSize,lotName,trianTestLabel,failureType,failureNum,trainTestNum
0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,[[Training]],[[none]],8,0
1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,[[Training]],[[none]],8,0
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,[[Training]],[[none]],8,0
3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,[[Training]],[[none]],8,0
4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,[[Training]],[[none]],8,0


In [8]:
wm811k['trianTestLabel'].apply(lambda x: str(x)).value_counts()

[]                638507
[['Test']]        118595
[['Training']]     54355
Name: trianTestLabel, dtype: int64

In [9]:
wm811k['trainTestNum'].apply(lambda x: str(x)).value_counts()

[]    638507
1     118595
0      54355
Name: trainTestNum, dtype: int64

### wm811k_test set
- 10,000개 augmentation된 데이터 train: validation : test = 65: 20: 15
- 최종 모델 확인 이후 wm811k_test 데이터 활용 모델 성능 확인하는 것이 좋을 듯

In [10]:
wm811k_test = wm811k.query("trainTestNum == 1")

In [11]:
wm811k_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118595 entries, 639663 to 811454
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   waferMap        118595 non-null  object 
 1   dieSize         118595 non-null  float64
 2   lotName         118595 non-null  object 
 3   trianTestLabel  118595 non-null  object 
 4   failureType     118595 non-null  object 
 5   failureNum      118595 non-null  object 
 6   trainTestNum    118595 non-null  object 
dtypes: float64(1), object(6)
memory usage: 7.2+ MB


In [12]:
wm811k_test['failureNum'].apply(lambda x: str(x)).value_counts()

8    110701
2      2772
4      1973
3      1126
0       832
6       693
5       257
1       146
7        95
Name: failureNum, dtype: int64

In [13]:
def zero_padding(data_img, set_size):
    height, width = data_img.shape
    
    if max(height, width) > set_size:
        return data_img
    
    delta_width = set_size - width
    delta_height = set_size - height
    top, bottom = delta_height//2, delta_height-(delta_height//2)
    left, right = delta_width//2, delta_width-(delta_width//2)
    
    padded_img = cv2.copyMakeBorder(data_img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0,0,0])
    return padded_img

## CNN-WDI Modeling

In [14]:
# CNN-WDI model
# layer별 padding option check ~ valid, same
input_shape = (224,224,1) # 학습 data shape 확인
class_num = 9
KERNEL_SIZE = 3

input_layer = layers.Input(shape=input_shape) # Input 224x224
x = layers.Conv2D(16, kernel_size=KERNEL_SIZE, padding='valid')(input_layer) # 16, 3x3
x = layers.Activation(activation='relu')(x) # ReLU
x = layers.BatchNormalization()(x) # BatchNormalization
x = layers.MaxPool2D(pool_size=(2,2))(x) # Max-pooling
x = layers.Conv2D(16, kernel_size=KERNEL_SIZE, padding='same')(x) # 16, 3x3
x = layers.Activation(activation='relu')(x) # ReLU
x = layers.BatchNormalization()(x) # BatchNormalization

x = layers.Conv2D(32, kernel_size=KERNEL_SIZE, padding='same')(x) # 32, 3x3
x = layers.Activation(activation='relu')(x) # ReLU
x = layers.BatchNormalization()(x) # BatchNormalization
x = layers.MaxPool2D(pool_size=(2,2))(x) # Max-pooling
x = layers.Conv2D(32, kernel_size=KERNEL_SIZE, padding='same')(x) # 32, 3x3
x = layers.Activation(activation='relu')(x) # ReLU
x = layers.BatchNormalization()(x) # BatchNormalization

x = layers.Conv2D(64, kernel_size=KERNEL_SIZE, padding='same')(x) # 64, 3x3
x = layers.Activation(activation='relu')(x) # ReLU
x = layers.BatchNormalization()(x) # BatchNormalization
x = layers.MaxPool2D(pool_size=(2,2))(x) # Max-pooling
x = layers.Conv2D(64, kernel_size=KERNEL_SIZE, padding='same')(x) # 64, 3x3
x = layers.Activation(activation='relu')(x) # ReLU
x = layers.BatchNormalization()(x) # BatchNormalization

x = layers.Conv2D(128, kernel_size=KERNEL_SIZE, padding='same')(x) # 128, 3x3
x = layers.Activation(activation='relu')(x) # ReLU
x = layers.BatchNormalization()(x) # BatchNormalization
x = layers.MaxPool2D(pool_size=(2,2))(x) # Max-pooling
x = layers.Conv2D(128, kernel_size=KERNEL_SIZE, padding='same')(x) # 128, 3x3
x = layers.Activation(activation='relu')(x) # ReLU
x = layers.BatchNormalization()(x) # BatchNormalization

x = layers.SpatialDropout2D(0.2)(x) # Spartial Dropout 0.2
x = layers.MaxPool2D(pool_size=(2,2))(x) # Max-pooling
x = layers.Flatten()(x) # 4608 확인
x = layers.Dense(512, activation='relu')(x)
output_layer = layers.Dense(class_num, activation='softmax')(x)

CNN_WDI = Model(input_layer, output_layer)

In [15]:
CNN_WDI.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 222, 222, 16)      160       
_________________________________________________________________
activation (Activation)      (None, 222, 222, 16)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 222, 222, 16)      64        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 111, 111, 16)      2320      
_________________________________________________________________
activation_1 (Activation)    (None, 111, 111, 16)      0     

In [16]:
# augmentation data load
wm811k_train_set = pd.read_pickle('./data/wm811k_train_augmentation.pkl')
wm811k_train_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90000 entries, 749336 to 699791
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   waferMap                   90000 non-null  object 
 1   dieSize                    90000 non-null  float64
 2   lotName                    90000 non-null  object 
 3   trianTestLabel             90000 non-null  object 
 4   failureType                90000 non-null  object 
 5   waferMapDim                90000 non-null  object 
 6   failureNum                 90000 non-null  object 
 7   trainTestNum               90000 non-null  object 
 8   waferMap_augmentation      90000 non-null  object 
 9   waferMap_augmentation_Dim  90000 non-null  object 
dtypes: float64(1), object(9)
memory usage: 7.6+ MB


In [17]:
wm811k_train_set.head(20)

,waferMap,dieSize,lotName,trianTestLabel,failureType,waferMapDim,failureNum,trainTestNum,waferMap_augmentation,waferMap_augmentation_Dim
749336,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 1, 2,...",518.0,lot45018,[[Training]],[[none]],"(27, 25)",8,0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","(224, 224)"
2087,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 2,...",533.0,lot84,[[Training]],[[none]],"(26, 26)",8,0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","(224, 224)"
242264,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",938.0,lot15080,[[Training]],[[none]],"(36, 34)",8,0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","(224, 224)"
710526,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2,...",516.0,lot43447,[[Training]],[[none]],"(25, 27)",8,0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","(224, 224)"
746385,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1,...",518.0,lot44899,[[Training]],[[none]],"(27, 25)",8,0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","(224, 224)"
765689,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1791.0,lot45680,[[Training]],[[none]],"(48, 48)",8,0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","(224, 224)"
3821,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2,...",533.0,lot154,[[Training]],[[none]],"(26, 26)",8,0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","(224, 224)"
705015,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2,...",516.0,lot43225,[[Training]],[[none]],"(25, 27)",8,0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","(224, 224)"
791314,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1534.0,lot46723,[[Training]],[[none]],"(45, 43)",8,0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","(224, 224)"
765130,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1791.0,lot45658,[[Training]],[[none]],"(48, 48)",8,0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","(224, 224)"


In [18]:
wm811k_train_set = wm811k_train_set.drop(['waferMap', 'dieSize', 'lotName', 'waferMapDim'], axis = 1)
wm811k_train_set.head()

,trianTestLabel,failureType,failureNum,trainTestNum,waferMap_augmentation,waferMap_augmentation_Dim
749336,[[Training]],[[none]],8,0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","(224, 224)"
2087,[[Training]],[[none]],8,0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","(224, 224)"
242264,[[Training]],[[none]],8,0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","(224, 224)"
710526,[[Training]],[[none]],8,0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","(224, 224)"
746385,[[Training]],[[none]],8,0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","(224, 224)"


In [19]:
wm811k_train_set = wm811k_train_set.reset_index(drop=True)
wm811k_train_set.head()

,trianTestLabel,failureType,failureNum,trainTestNum,waferMap_augmentation,waferMap_augmentation_Dim
0,[[Training]],[[none]],8,0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","(224, 224)"
1,[[Training]],[[none]],8,0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","(224, 224)"
2,[[Training]],[[none]],8,0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","(224, 224)"
3,[[Training]],[[none]],8,0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","(224, 224)"
4,[[Training]],[[none]],8,0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","(224, 224)"


- 참고 : https://www.kaggle.com/code/yashsawarn/wm-811k-wafermap/notebook

In [ ]:
# wm811k_X_train_set_reshape : 파일로 저장하기
wm811k_X_train_set_reshape =  np.load('D:/capstone_data/wm811k_train_augmentation_reshape.npy')
wm811k_y_train_set = wm811k_train_set['failureNum']

In [ ]:
save_test_none = wm811k_X_train_set_reshape[500] # wm811k_new_train_class_None,
save_test_edge_ring = wm811k_X_train_set_reshape[10500] # wm811k_new_train_class_Edge_Ring_augmentation,
save_test_center = wm811k_X_train_set_reshape[20500] # wm811k_new_train_class_Center_augmentation,
save_test_edge_loc = wm811k_X_train_set_reshape[30500] # wm811k_new_train_class_Edge_Loc_augmentation,
save_test_loc = wm811k_X_train_set_reshape[40500] # wm811k_new_train_class_Loc_augmentation,
save_test_random = wm811k_X_train_set_reshape[50500] # wm811k_new_train_class_Random_augmentation,
save_test_scratch = wm811k_X_train_set_reshape[60500] # wm811k_new_train_class_Scratch_augmentation,
save_test_donut = wm811k_X_train_set_reshape[70500] # wm811k_new_train_class_Donut_augmentation,
save_test_near_full = wm811k_X_train_set_reshape[80500] # wm811k_new_train_class_Near_Full_augmentation

plt.imshow(save_test_none)
plt.show

plt.imshow(save_test_edge_ring)
plt.show

plt.imshow(save_test_center)
plt.show

plt.imshow(save_test_edge_loc)
plt.show

plt.imshow(save_test_loc)
plt.show

plt.imshow(save_test_random)
plt.show

plt.imshow(save_test_scratch)
plt.show

plt.imshow(save_test_donut)
plt.show

plt.imshow(save_test_near_full)
plt.show

In [ ]:
# train / validation / test set 설정
# 60 / 20 / 20
X_train, X_test, y_train, y_test = train_test_split(wm811k_X_train_set_reshape, wm811k_y_train_set, test_size=0.2, random_state=2022)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=2022)  # 20/80 = 0.25

In [ ]:
# model_id
model_id = 'cnn_wdi_{0}'.format(datetime.datetime.now())

# model 설정
LEARNING_RATE = 0.001
BATCH_SIZE = 100
EPOCH = 20  # 논문 그래프에 20 epoch 까지 있음

CNN_WDI.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=LEARNING_RATE), metrics=['categorical_crossentropy'])
checkpointer = ModelCheckpoint(filepath="{0}.h5".format(model_id), verbose=1, save_best_only=True)

# 학습 실행
hist = CNN_WDI.fit(X_train, y_train, epochs=EPOCH, batch_size=BATCH_SIZE, shuffle=True,
                   validation_data=(X_val, y_val), callbacks=[checkpointer])


In [ ]:
score = CNN_WDI.evaluate(X_test, y_test, verbose=0)
print('Test Loss : ', score[0])
print('Test Accuracy : ', score[1])

In [ ]:
# 원본 test data로 confusion matrix
# 원본 test 데이터 padding 하는 과정 필요